In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("Datos_limpios_Berlin.csv")
df.head(5)

,Unnamed: 0,last_scraped,name,description,neighborhood_overview,picture_url,host_name,host_since,host_about,host_response_time,...,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,0,18/09/2024,130 sqm with a view on Mauer Park,An apartment of 130 sqm right in the heart of ...,Prenzlauer Berg is no doubt one of the most my...,https://a0.muscache.com/pictures/miso/Hosting-...,Marine,14/11/2009,I'm french but I have been living in Berlin si...,within an hour,...,15.0,0.0,4.93,4.95,4.90,4.97,4.94,4.82,4.76,0.80
1,1,18/09/2024,light quiet beautiful 4 room flat,We offer our 4 room apartment in Prenzlauer Be...,Prenzlauer Berg is no doubt one of the most my...,https://a0.muscache.com/pictures/caa048ba-e297...,Katja,08/11/2010,Designer who likes discovering the world,within a few hours,...,7.0,1.0,4.84,4.88,4.79,4.81,4.91,4.87,4.73,0.43
2,2,18/09/2024,Super Apartment in Berlin/Kreuzberg,Our fantastic typical Berlin-style apartment i...,Prenzlauer Berg is no doubt one of the most my...,https://a0.muscache.com/pictures/176e2964-5ead...,Guidi,08/09/2010,"Hi , I am Elena and I am looking forward to ho...",within a few hours,...,4.0,0.0,4.70,4.68,4.65,4.82,4.82,4.94,4.47,0.20
3,3,18/09/2024,"Rooftop apt w/ a view, 2 bedrooms/bathrooms, s...","I am renting out my spacious, bright rooftop a...","Kastanienalle, Oderberger Strasse, Hackescher ...",https://a0.muscache.com/pictures/1135114/9db3c...,Stephan,08/12/2010,Ich mag sch‚àö‚àÇne Dinge und Design. Wenn es ...,within a day,...,0.0,0.0,4.75,4.63,4.50,4.94,4.63,4.88,4.40,0.10
4,4,18/09/2024,Room w/ balcony and private bathroom in nice area,I¬¨¬•m offering a guest room with balcony in m...,"The neighborhood ""Prenzlauer Berg"" is centrall...",https://a0.muscache.com/pictures/miso/Hosting-...,Christian,07/12/2010,"Christian from Berlin\n\nHi, I¬¨¬•m a 52 years...",within a few hours,...,22.0,0.3,4.95,4.95,4.92,4.95,4.92,4.84,4.77,0.73


In [3]:
df= df.drop(['Unnamed: 0'], axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11527 entries, 0 to 11526
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   last_scraped                 11527 non-null  object 
 1   name                         11527 non-null  object 
 2   description                  11527 non-null  object 
 3   neighborhood_overview        11527 non-null  object 
 4   picture_url                  11527 non-null  object 
 5   host_name                    11527 non-null  object 
 6   host_since                   11527 non-null  object 
 7   host_about                   11527 non-null  object 
 8   host_response_time           11527 non-null  object 
 9   host_is_superhost            11527 non-null  object 
 10  host_verifications           11527 non-null  object 
 11  host_has_profile_pic         11527 non-null  object 
 12  host_identity_verified       11527 non-null  object 
 13  neighbourhood   

In [5]:
valores_nulos=df.isnull().sum()
valores_nulos

last_scraped                   0
name                           0
description                    0
neighborhood_overview          0
picture_url                    0
host_name                      0
host_since                     0
host_about                     0
host_response_time             0
host_is_superhost              0
host_verifications             0
host_has_profile_pic           0
host_identity_verified         0
neighbourhood                  0
property_type                  0
room_type                      0
bathrooms_text                 0
amenities                      0
calendar_updated               0
has_availability               0
calendar_last_scraped          0
last_review                    0
license                        0
instant_bookable               0
id                             0
host_response_rate             0
host_acceptance_rate           0
host_listings_count            0
host_total_listings_count      0
accommodates                   0
bathrooms 

In [6]:
# Convertir host_response_time a binario
df["host_response_time"] = df["host_response_time"].replace({
    "within an hour": "Rápido",
    "within a few hours": "Rápido",
    "within a day": "Lento",
    "a few days or more": "Lento",
    "SIN INFORMACION": "Lento"
})

# Verificar que la conversión fue correcta
print(df["host_response_time"].value_counts())

host_response_time
Lento     6045
Rápido    5482
Name: count, dtype: int64


In [8]:
df["host_is_superhost"] = df["host_is_superhost"].replace({"t": 1, "f": 0})
df["host_is_superhost"] = df["host_is_superhost"].replace({"SIN INFORMACION": 0})

In [9]:
vars_indep=df[['host_acceptance_rate', 'host_is_superhost', 'number_of_reviews']]
var_dep=df['host_response_time']

In [10]:
X=vars_indep
y=var_dep

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [12]:
escalar=StandardScaler()

In [13]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [14]:
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

In [15]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [16]:
y_pred=algoritmo.predict(X_test)
y_pred

array(['Lento', 'Lento', 'Rápido', ..., 'Lento', 'Rápido', 'Rápido'],
      dtype=object)

In [17]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[1628  201]
 [ 709  921]]


In [18]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="Rápido")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.820855614973262


In [19]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7369181844463718


In [20]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="Rápido")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.5650306748466257


In [21]:
df["host_is_superhost"] = df["host_is_superhost"].replace({1:"t", 0:"f"})

In [22]:
vars_indep=df[['host_response_rate', 'host_acceptance_rate', 'host_total_listings_count']]
var_dep=df['host_is_superhost']

X=vars_indep
y=var_dep

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], dtype=object)

In [24]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[2668    2]
 [ 784    5]]


In [25]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.7728852838933952


In [26]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7727666955767563


In [27]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9992509363295881


In [28]:
df["room_type"] = df["room_type"].replace({"Entire home/apt": 0, "Private room": 1, "Hotel room":2, "Shared room":3})

C:\Users\angie\AppData\Local\Temp\ipykernel_8620\1760127900.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["room_type"] = df["room_type"].replace({"Entire home/apt": 0, "Private room": 1, "Hotel room":2, "Shared room":3})


In [29]:
df["beds"] = df["beds"].replace({
    0: "De 0 a 2 camas",
    1: "De 0 a 2 camas",
    1.7: "De 0 a 2 camas",
    2: "De 2 a 3 camas",
    3: "De 2 a 3 camas"
})

# Verificar que la conversión fue correcta
print(df["beds"].value_counts())

beds
De 2 a 3 camas    7208
De 0 a 2 camas    4319
Name: count, dtype: int64


In [30]:
vars_indep=df[['accommodates', 'bedrooms', 'room_type']]
var_dep=df['beds']

X=vars_indep
y=var_dep

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['De 2 a 3 camas', 'De 2 a 3 camas', 'De 2 a 3 camas', ...,
       'De 2 a 3 camas', 'De 2 a 3 camas', 'De 2 a 3 camas'], dtype=object)

In [32]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[   0 1313]
 [   0 2146]]


In [37]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="De 0 a 2 camas")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.0


c:\Users\angie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.6204105232726221


In [39]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="De 0 a 2 camas")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0


In [40]:
df["host_is_superhost"] = df["host_is_superhost"].replace({"t": 1, "f": 0, "SIN INFORMACION":0})
df["host_verifications"] = df["host_verifications"].replace({
    "[]": 0, 
    "['phone']": 1, 
    "['email']": 2, 
    "['phone', 'work_email']": 3,
    "['email', 'work_email']": 4,
    "['email', 'phone']": 5,
    "['email', 'phone', 'work_email']": 6
})

C:\Users\angie\AppData\Local\Temp\ipykernel_8620\2607528318.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["host_is_superhost"] = df["host_is_superhost"].replace({"t": 1, "f": 0, "SIN INFORMACION":0})
C:\Users\angie\AppData\Local\Temp\ipykernel_8620\2607528318.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["host_verifications"] = df["host_verifications"].replace({


In [41]:
vars_indep=df[['host_listings_count', 'host_verifications', 'host_is_superhost']]
var_dep=df['host_has_profile_pic']

X=vars_indep
y=var_dep

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [43]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[   0   51]
 [   0 3408]]


In [44]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9852558542931483


In [45]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9852558542931483


In [46]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


In [47]:
vars_indep=df[['host_is_superhost', 'review_scores_communication', 'review_scores_rating']]
var_dep=df['host_identity_verified']

In [48]:
X=vars_indep
y=var_dep

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [50]:
escalar=StandardScaler()

In [51]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [52]:
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

In [53]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [54]:
y_pred=algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [55]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[   0  313]
 [   0 3146]]


In [56]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9095114194854004


In [57]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9095114194854004


In [58]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


In [3]:
df["property_type"] = df["property_type"].replace({
    "Entire rental unit": 1,
    "Private room in rental unit": 2,
    "Entire condo": 3,
    "Entire loft": 4,
    "Entire guest suite": 5,
    "Entire home": 6,
    "Private room in loft": 7,
    "Private room in condo": 8,
    "Private room in home": 9,
    "Private room": 10,
    "Entire townhouse": 11,
    "Private room in hostel": 12,
    "Shared room in hostel": 13,
    "Entire place": 14,
    "Entire guesthouse": 15,
    "Houseboat": 16,
    "Private room in bed and breakfast": 17,
    "Shared room in rental unit": 18,
    "Entire bungalow": 19,
    "Boat": 20,
    "Room in aparthotel": 21,
    "Entire cottage": 22,
    "Private room in townhouse": 23,
    "Entire villa": 24,
    "Entire serviced apartment": 25,
    "Room in boutique hotel": 26,
    "Tiny home": 27,
    "Shared room in boat": 28,
    "Private room in villa": 29,
    "Private room in tipi": 30,
    "Private room in guest suite": 31,
    "Room in hotel": 32,
    "Room in hostel": 33,
    "Private room in guesthouse": 34,
    "Room in serviced apartment": 35,
    "Private room in cave": 36,
    "Entire chalet": 37,
    "Entire vacation home": 38,
    "Private room in bungalow": 39,
    "Camper/RV": 40,
    "Private room in pension": 41,
    "Private room in serviced apartment": 42,
    "Shared room in boutique hotel": 43,
    "Shared room in loft": 44,
    "Island": 45,
    "Entire cabin": 46,
    "Private room in boat": 47,
    "Private room in casa particular": 48,
    "Private room in houseboat": 49,
    "Private room in vacation home": 50,
    "Cave": 51,
    "Shared room": 52,
    "Shared room in serviced apartment": 53,
    "Private room in castle": 54,
    "Private room in shipping container": 55,
    "Shared room in bed and breakfast": 56,
    "Shared room in hotel": 57,
    "Dome": 58,
    "Casa particular": 59,
    "Private room in chalet": 60
})


C:\Users\angie\AppData\Local\Temp\ipykernel_12376\144347394.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["property_type"] = df["property_type"].replace({


In [4]:
df["room_type"] = df["room_type"].replace({
    "Entire home/apt":"Entire home/apt ",
    "Private room":"No Entire home/apt",
    "Shared room":"No Entire home/apt",
    "Hotel room":"No Entire home/apt"
})

print(df["room_type"].value_counts())

room_type
Entire home/apt       7593
No Entire home/apt    3934
Name: count, dtype: int64


In [5]:
vars_indep=df[['property_type', 'accommodates', 'price']]
var_dep=df['room_type']

X=vars_indep
y=var_dep

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['Entire home/apt ', 'Entire home/apt ', 'Entire home/apt ', ...,
       'Entire home/apt ', 'Entire home/apt ', 'Entire home/apt '],
      dtype=object)

In [7]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[2080  175]
 [ 717  487]]


In [8]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="No Entire home/apt")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.7356495468277946


In [9]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7421220005782018


In [10]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="No Entire home/apt")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.40448504983388706


In [11]:
df["host_is_superhost"] = df["host_is_superhost"].replace({"SIN INFORMACION":'f'})

In [12]:
df["has_availability"] = df["has_availability"].replace({"SIN INFORMACION":'f'})

In [13]:
vars_indep=df[['price', 'availability_365', 'number_of_reviews']]
var_dep=df['has_availability']

X=vars_indep
y=var_dep

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [15]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[   0    5]
 [   0 3454]]


In [16]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9985544955189362


In [17]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9985544955189362


In [18]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


In [19]:
df["host_response_time"] = df["host_response_time"].replace({
    "within an hour": 1,
    "within a few hours": 2,
    "within a day": 3,
    "a few days or more": 4,
    "SIN INFORMACION": 5
})

# Verificar que la conversión fue correcta
print(df["host_response_time"].value_counts())

host_response_time
3    5687
1    4253
2    1229
4     358
Name: count, dtype: int64


C:\Users\angie\AppData\Local\Temp\ipykernel_12376\2354697535.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["host_response_time"] = df["host_response_time"].replace({


In [20]:
vars_indep=df[['maximum_nights', 'minimum_nights', 'host_response_time']]
var_dep=df['instant_bookable']

X=vars_indep
y=var_dep

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['f', 't', 'f', ..., 'f', 't', 'f'], dtype=object)

In [22]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[2452  154]
 [ 679  174]]


In [23]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.7831363781539444


In [24]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7591789534547557


In [25]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9409056024558711


In [3]:
df["room_type"] = df["room_type"].replace({
    "Entire home/apt": 1,
    "Private room": 2,
    "Shared room": 3,
    "Hotel room": 4
})

print(df["room_type"].value_counts())

room_type
1    7593
2    3698
3     130
4     106
Name: count, dtype: int64


C:\Users\angie\AppData\Local\Temp\ipykernel_19808\4175036345.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["room_type"] = df["room_type"].replace({


In [ ]:
df["bedrooms"] = df["bedrooms"].replace({
    1: "Hay 1 recamara"
})

# Verificar que la conversión fue correcta
print(df["bedrooms"].value_counts())

bedrooms
Hay 1 recamara    11527
Name: count, dtype: int64


In [6]:
vars_indep=df[['room_type', 'bathrooms', 'beds']]
var_dep=df['bedrooms']

X=vars_indep
y=var_dep

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 'Hay 1 recamara'

In [16]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[   0   50]
 [   0 3409]]


In [ ]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="De 0 a 1 recamara")
print('Precision del modelo:')
print(precision)

In [ ]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

In [ ]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="De 0 a 1 recamara")
print('Sensibilidad del modelo:')
print(sensibilidad)

In [8]:
df["minimum_nights"] = df["minimum_nights"].replace({
    1: "Pocas noches mínimas",
    1.6: "Pocas noches mínimas",
    2: "Pocas noches mínimas",
    3: "Pocas noches mínimas",
    4: "Pocas noches mínimas",
    5: "Pocas noches mínimas",
    6: "Pocas noches mínimas",
    7: "Pocas noches mínimas",
    8: "Pocas noches mínimas",
    9: "Pocas noches mínimas",
    10: "Pocas noches mínimas",
    11: "Pocas noches mínimas",
    12: "Pocas noches mínimas",
    13: "Pocas noches mínimas",
    14: "Pocas noches mínimas",
    15: "Pocas noches mínimas",
    16: "Pocas noches mínimas",
    17: "Pocas noches mínimas",
    18: "Pocas noches mínimas",
    19: "Pocas noches mínimas",
    20: "Pocas noches mínimas",
    21: "Pocas noches mínimas",
    22: "Pocas noches mínimas",
    24: "Pocas noches mínimas",
    25: "Pocas noches mínimas",
    26: "Pocas noches mínimas",
    27: "Pocas noches mínimas",
    28: "Pocas noches mínimas",
    29: "Pocas noches mínimas",
    30: "Pocas noches mínimas",
    31: "Pocas noches mínimas",
    36: "Pocas noches mínimas",
    40: "Pocas noches mínimas",
    42.9: "Pocas noches mínimas",
    45: "Pocas noches mínimas",
    50: "Pocas noches mínimas",
    54: "Pocas noches mínimas",
    55: "Pocas noches mínimas",
    56: "Pocas noches mínimas",
    57: "Pocas noches mínimas",
    60: "Pocas noches mínimas",
    62: "Pocas noches mínimas",
    63: "Pocas noches mínimas",
    70: "Pocas noches mínimas",
    75: "Pocas noches mínimas",
    80: "Pocas noches mínimas",
    82: "Pocas noches mínimas",
    89: "Pocas noches mínimas",
    90: "Pocas noches mínimas",
    91: "Pocas noches mínimas",
    92: "Pocas noches mínimas",
    93: "Pocas noches mínimas",
    94: "Pocas noches mínimas",
    95: "Pocas noches mínimas",
    96: "Pocas noches mínimas",
    97: "Pocas noches mínimas",
    98: "Pocas noches mínimas",
    99: "Pocas noches mínimas",
    100: "Pocas noches mínimas",
    105: "Muchas noches mínimas",
    110: "Muchas noches mínimas",
    111: "Muchas noches mínimas",
    119: "Muchas noches mínimas",
    120: "Muchas noches mínimas",
    121: "Muchas noches mínimas",
    124: "Muchas noches mínimas",
    129: "Muchas noches mínimas",
    130: "Muchas noches mínimas",
    131: "Muchas noches mínimas",
    140: "Muchas noches mínimas",
    150: "Muchas noches mínimas",
    155: "Muchas noches mínimas",
    165: "Muchas noches mínimas",
    168: "Muchas noches mínimas",
    170: "Muchas noches mínimas",
    179: "Muchas noches mínimas",
    180: "Muchas noches mínimas",
    181: "Muchas noches mínimas",
    182: "Muchas noches mínimas",
    183: "Muchas noches mínimas",
    184: "Muchas noches mínimas",
    185: "Muchas noches mínimas",
    186: "Muchas noches mínimas",
    187: "Muchas noches mínimas",
    190: "Muchas noches mínimas",
    199: "Muchas noches mínimas",
    200: "Muchas noches mínimas"
})

# Verificar que la conversión fue correcta
print(df["minimum_nights"].value_counts())


minimum_nights
Pocas noches mínimas     11350
Muchas noches mínimas      177
Name: count, dtype: int64


In [9]:
vars_indep=df[['price', 'room_type', 'availability_365']]
var_dep=df['minimum_nights']

X=vars_indep
y=var_dep

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
escalar=StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred=algoritmo.predict(X_test)
y_pred

array(['Pocas noches mínimas', 'Pocas noches mínimas',
       'Pocas noches mínimas', ..., 'Pocas noches mínimas',
       'Pocas noches mínimas', 'Pocas noches mínimas'], dtype=object)

In [11]:
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print('Matriz de confusion:')
print(matriz)

Matriz de confusion:
[[   0   50]
 [   0 3409]]


In [13]:
from sklearn.metrics import precision_score

precision=precision_score(y_test, y_pred, average="binary", pos_label="Pocas noches mínimas")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9855449551893611


In [14]:
from sklearn.metrics import accuracy_score

exactitud=accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9855449551893611


In [15]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="Pocas noches mínimas")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0
